# HAPI‑FHIR + Synthea Sandbox (PowerShell Edition)

This notebook reproduces the entire setup and validation workflow we walked through in chat, but every shell cell is written for **PowerShell** instead of Git Bash.  Run the cells step‑by‑step in VS Code’s Jupyter view (kernel: *Python* is fine because each shell cell is self‑contained).

## Prerequisites
* Docker Desktop running locally
* Synthea export folder at `D:\synthea_out\fhir` (one **transaction Bundle** JSON per patient)
* A PowerShell‑enabled Jupyter environment (VS Code does this automatically when you prefix cells with `%%powershell`)

## 1  Start (or restart) the HAPI‑FHIR server container

In [ ]:
import subprocess, sys

ps_script = r"""
Write-Output "Stopping and removing current HAPI FHIR container (ignore errors if it doesn't exist)..."
docker stop hapi 2>$null
docker rm   hapi 2>$null

Write-Output "Dropping old volume to reset schema..."
docker volume rm hapi-data 2>$null

Write-Output "Starting HAPI FHIR container (heap 18 GB, cgroup 22 GB)..."
docker run -d --name hapi `
  -p 8080:8080 `
  -v hapi-data:/data/hapi `
  --memory=24g --cpus=6 `
  -e spring.datasource.url=jdbc:h2:file:/data/hapi/h2db `
  -e spring.datasource.driverClassName=org.h2.Driver `
  -e hapi.fhir.scheduler.enabled=false `
  -e JAVA_OPTS="-Xmx20g -XX:+UseG1GC" `
  hapiproject/hapi:latest
"""

print("Launching PowerShell…")
completed = subprocess.run(
    ["powershell", "-NoLogo", "-NoProfile", "-Command", ps_script],
    text=True, capture_output=True
)

print(completed.stdout)
if completed.stderr:
    print("STDERR:\n", completed.stderr, file=sys.stderr)


Launching PowerShell…



STDERR:
 At line:13 char:5
+   --memory=24g --cpus=6 \
+     ~
Missing expression after unary operator '--'.
At line:13 char:5
+   --memory=24g --cpus=6 \
+     ~~~~~~~~~~
Unexpected token 'memory=24g' in expression or statement.
    + CategoryInfo          : ParserError: (:) [], ParentContainsErrorRecordException
    + FullyQualifiedErrorId : MissingExpressionAfterOperator
 



Not applying following code, for postgreps SQL is not supported well by the hapi image, the deep reason is that in order to let JDBC execute SQL, it need HFJ_RESOURCE, so I need to Run Flyway migrations, However, the Flyway integreated in this hapi image do not support any version of PostgrepsSQL, any version of hapi image's flyway do not support any version of postgres image. (Caused by: org.springframework.beans.factory.BeanCreationException: Error creating bean with name 'flywayInitializer' defined in class path resource [org/springframework/boot/autoconfigure/flyway/FlywayAutoConfiguration$FlywayConfiguration.class]: Unsupported Database: PostgreSQL XX.XX
2025-07-02 23:54:42)
Use docker-compose.yml to create docker in order to use postgres rather than H2 to store the data to prevent Out of Memory Error.

## 2  Set local variables

In [26]:
import subprocess

# Define paths and URL
powershell_script = """
$BUNDLE_DIR = '//Desktop-family/K/synthea_out/fhir'  # Updated to match the path in prerequisites
$FHIR_BASE  = "http://localhost:8080/fhir/"
Write-Host "BUNDLE_DIR set to: $BUNDLE_DIR"
Write-Host "FHIR_BASE set to: $FHIR_BASE"

# Set these variables as environment variables so they persist for future cells
[Environment]::SetEnvironmentVariable("BUNDLE_DIR", $BUNDLE_DIR, "Process")
[Environment]::SetEnvironmentVariable("FHIR_BASE", $FHIR_BASE, "Process")
"""

# Execute PowerShell script
result = subprocess.run(["powershell", "-Command", powershell_script], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)

# Store in Python variables for potential use in Python cells
BUNDLE_DIR = '//Desktop-family/K/synthea_out/fhir'
FHIR_BASE = "http://localhost:8080/fhir/"

BUNDLE_DIR set to: //Desktop-family/K/synthea_out/fhir
FHIR_BASE set to: http://localhost:8080/fhir/



### Health check: CapabilityStatement should return JSON

In [32]:
import subprocess

# PowerShell script to check FHIR server health with explicit URL
ps_script = """
$FHIR_BASE = "http://localhost:8080/fhir/"
Invoke-RestMethod -Uri ($FHIR_BASE + "metadata") | ConvertTo-Json -Depth 1 | Select-Object -First 20
"""

# Execute PowerShell script
result = subprocess.run(["powershell", "-Command", ps_script], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)


Error: Invoke-RestMethod : Unable to connect to the remote server
At line:3 char:1
+ Invoke-RestMethod -Uri ($FHIR_BASE + "metadata") | ConvertTo-Json -De ...
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : InvalidOperation: (System.Net.HttpWebRequest:HttpWebRequest) [Invoke-RestMethod], WebExc 
   eption
    + FullyQualifiedErrorId : WebCmdletWebResponseException,Microsoft.PowerShell.Commands.InvokeRestMethodCommand
 



## 3  Upload every Synthea transaction bundle

1. Verify Docker container is running properly


In [ ]:
import subprocess

ps_script = """
# Check if HAPI FHIR container is running
Write-Output "Checking Docker container status:"
docker ps -a | Select-String -Pattern "hapi"

# Check container logs for errors
Write-Output "`nContainer logs (last 20 lines):"
docker logs --tail 20 hapi
"""

result = subprocess.run(["powershell", "-Command", ps_script], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)

Checking Docker container status:

85acf0d7d13a   hapiproject/hapi:latest   "java --class-path /…"   42 seconds ago   Up 42 seconds   
0.0.0.0:8080->8080/tcp, [::]:8080->8080/tcp   hapi

Container logs (last 20 lines):
2025-07-02T13:49:45.913Z  INFO 1 --- [           main] c.u.f.j.sched.BaseSchedulerServiceImpl   : Scheduling local job ca.uhn.fhir.jpa.cache.ResourceChangeListenerCacheRefresherImpl with interval 00:00:10.000
2025-07-02T13:49:45.913Z  INFO 1 --- [           main] c.u.f.j.sched.BaseSchedulerServiceImpl   : Scheduling clustered job ca.uhn.fhir.jpa.search.reindex.ResourceReindexingSvcImpl with interval 00:00:10.000
2025-07-02T13:49:45.914Z  INFO 1 --- [           main] c.u.f.j.sched.BaseSchedulerServiceImpl   : Scheduling clustered job ca.uhn.fhir.jpa.bulk.export.svc.BulkDataExportJobSchedulingHelperImpl$PurgeExpiredFilesJob with interval 01:00:00
2025-07-02T13:49:45.914Z  INFO 1 --- [           main] c.u.f.j.sched.BaseSchedulerServiceImpl   : Scheduling local job ca.uhn.fh

2. Test basic connectivity to the FHIR server


In [8]:
import subprocess

ps_script = """
Write-Output "Testing basic connectivity to FHIR server..."
try {
    $response = Invoke-WebRequest -Uri "http://localhost:8080/fhir/metadata" -TimeoutSec 5
    Write-Output "Connection successful! Status code: $($response.StatusCode)"
} catch {
    Write-Output "Failed to connect: $_"
    
    # Check if service is listening on port
    Write-Output "`nChecking port 8080:"
    netstat -ano | findstr :8080
}
"""

result = subprocess.run(["powershell", "-Command", ps_script], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)

Testing basic connectivity to FHIR server...
Connection successful! Status code: 200



3. Wait for server to initialize (this is critical):


In [3]:
import subprocess
import time

print("Waiting for HAPI FHIR server to initialize...")
ps_script = """
$maxAttempts = 12
$attempt = 0
$success = $false

while (-not $success -and $attempt -lt $maxAttempts) {
    $attempt++
    Write-Output "Connection attempt $attempt of $maxAttempts..."
    
    try {
        $response = Invoke-WebRequest -Uri "http://localhost:8080/fhir/metadata" -TimeoutSec 10
        if ($response.StatusCode -eq 200) {
            Write-Output "Server is up and running! Status code: $($response.StatusCode)"
            $success = $true
        }
    } catch {
        Write-Output "Still waiting for server... ($($_.Exception.Message))"
        Start-Sleep -Seconds 10
    }
}

if (-not $success) {
    Write-Output "Server did not become available after $(10*$maxAttempts) seconds"
} else {
    Write-Output "HAPI FHIR server is ready to use"
}
"""

result = subprocess.run(["powershell", "-Command", ps_script], capture_output=True, text=True)
print(result.stdout)

Waiting for HAPI FHIR server to initialize...
Connection attempt 1 of 12...
Still waiting for server... (The operation has timed out.)
Connection attempt 2 of 12...
Still waiting for server... (The operation has timed out.)
Connection attempt 3 of 12...
Still waiting for server... (The operation has timed out.)
Connection attempt 4 of 12...
Still waiting for server... (The operation has timed out.)
Connection attempt 5 of 12...
Still waiting for server... (The operation has timed out.)
Connection attempt 6 of 12...
Still waiting for server... (The operation has timed out.)
Connection attempt 7 of 12...
Still waiting for server... (The operation has timed out.)
Connection attempt 8 of 12...
Still waiting for server... (The operation has timed out.)
Connection attempt 9 of 12...
Still waiting for server... (The operation has timed out.)
Connection attempt 10 of 12...
Still waiting for server... (The operation has timed out.)
Connection attempt 11 of 12...
Still waiting for server... (The

pwsh -ExecutionPolicy Bypass -File .\upload_fhir_files.ps1 |
  Tee-Object -FilePath upload_log.txt

5. Testing if a HAPI FHIR Server is Running Correctly


In [16]:
import subprocess, sys

ps_script = r"""
$fhirBase = "http://localhost:8080/fhir"
$results = [ordered]@{}

Write-Output "========== HAPI FHIR SERVER HEALTH CHECK =========="

# 1. Basic connectivity test
Write-Output "`n1. Testing basic connectivity..."
try {
    $response = Invoke-WebRequest -Uri "$fhirBase/metadata" -TimeoutSec 5 -UseBasicParsing
    $results["Basic Connectivity"] = "OK (Status: $($response.StatusCode))"
    Write-Output "Server is reachable! Status code: $($response.StatusCode)"
} catch {
    $results["Basic Connectivity"] = "FAILED: $($_.Exception.Message)"
    Write-Output "Failed to connect: $($_.Exception.Message)"
}
"""

print("Running comprehensive HAPI FHIR server health check...")
result = subprocess.run(["powershell", "-NoLogo", "-NoProfile", "-Command", ps_script], 
                        capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr, file=sys.stderr)

Running comprehensive HAPI FHIR server health check...
========== HAPI FHIR SERVER HEALTH CHECK ==========

1. Testing basic connectivity...
   Server is reachable! Status code: 200



In [17]:

# 2. Check server version and capabilities
import subprocess, sys

ps_script = r"""
$fhirBase = "http://localhost:8080/fhir"
$results = [ordered]@{}

Write-Output "========== HAPI FHIR SERVER HEALTH CHECK =========="
try {
    $metadata = Invoke-RestMethod -Uri "$fhirBase/metadata" -TimeoutSec 10
    $version = $metadata.software.version
    $fhirVersion = $metadata.fhirVersion
    $results["Server Version"] = $version
    $results["FHIR Version"] = $fhirVersion
    
    Write-Output "   Server info: HAPI FHIR $version (FHIR $fhirVersion)"
    
    # List supported resource types
    $resourceTypes = $metadata.rest.resource.type | Sort-Object -Unique
    $results["Resource Types"] = "$($resourceTypes.Count) types supported"
    Write-Output "   Supports $($resourceTypes.Count) resource types"
    
    # Check if common resources are supported
    $commonTypes = @("Patient", "Observation", "Encounter", "Condition")
    foreach ($type in $commonTypes) {
        if ($resourceTypes -contains $type) {
            Write-Output "   Supports $type resources"
        } else {
            Write-Output "Does NOT support $type resources"
        }
    }
    
    # List operations
    $operations = $metadata.rest.operation.name | Sort-Object -Unique
    if ($operations) {
        $results["Operations"] = "$($operations.Count) operations supported"
        Write-Output "   Supports $($operations.Count) operations: $($operations -join ', ')"
    } else {
        $results["Operations"] = "None found"
        Write-Output "No operations found in capabilities statement"
    }
    
} catch {
    $results["Server Capabilities"] = " FAILED: $($_.Exception.Message)"
    Write-Output "   Failed to retrieve capabilities: $($_.Exception.Message)"
}
"""

print("Running comprehensive HAPI FHIR server health check...")
result = subprocess.run(["powershell", "-NoLogo", "-NoProfile", "-Command", ps_script], 
                        capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr, file=sys.stderr)
    


Running comprehensive HAPI FHIR server health check...
========== HAPI FHIR SERVER HEALTH CHECK ==========
   Server info: HAPI FHIR 8.2.0 (FHIR 4.0.1)
   Supports 146 resource types
   Supports Patient resources
   Supports Observation resources
   Supports Encounter resources
   Supports Condition resources
   Supports 9 operations: diff, expunge, get-resource-counts, hapi.fhir.replace-references, mark-all-resources-for-reindexing, meta, perform-reindexing-pass, reindex, reindex-terminology



## 4  Validation suite

# 4.1 Resource counts

In [18]:
# 3. Resource counts test
import subprocess, sys

ps_script = r"""
$fhirBase = "http://localhost:8080/fhir"
$results = [ordered]@{}

Write-Output "`n3. Checking for existing resources..."
$resourceTypes = @(
    "Patient", "Encounter", "Observation", "Condition", 
    "Procedure", "MedicationRequest", "Immunization"
)
$resourceCounts = @{}

foreach ($type in $resourceTypes) {
    try {
        $response = Invoke-RestMethod -Uri "$fhirBase/$type`?_summary=count" -Method Get -TimeoutSec 15
        $count = $response.total
        $resourceCounts[$type] = $count
        
        if ($count -gt 0) {
            Write-Output "   Found $count $type resources"
        } else {
            Write-Output "No $type resources found"
        }
    } catch {
        $errorMessage = $_.Exception.Message
        $resourceCounts[$type] = "Error: $errorMessage"
        Write-Output "   Error checking $type - $errorMessage"
    }
}

$results["Resource Counts"] = $resourceCounts
"""

print("Running comprehensive HAPI FHIR server health check...")
result = subprocess.run(["powershell", "-NoLogo", "-NoProfile", "-Command", ps_script], 
                        capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr, file=sys.stderr)
    


Running comprehensive HAPI FHIR server health check...

3. Checking for existing resources...
   Found 404 Patient resources
   Found 14346 Encounter resources
   Found 77224 Observation resources
   Found 10365 Condition resources
   Found 40430 Procedure resources
   Found 5907 MedicationRequest resources
   Found 6011 Immunization resources



# 4.2 Test a sample Patient query

In [ ]:
# 4. Test a sample Patient query
import subprocess, sys

ps_script = r"""
$fhirBase = "http://localhost:8080/fhir"
$results = [ordered]@{}

Write-Output "`n4. Testing Patient query..."
try {
    $patients = Invoke-RestMethod -Uri "$fhirBase/Patient?_count=1" -TimeoutSec 15
    
    if ($patients.entry -and $patients.entry.Count -gt 0) {
        $patientId = $patients.entry[0].resource.id
        $results["Sample Patient"] = "Found (ID: $patientId)"
        Write-Output "   Successfully retrieved patient with ID: $patientId"
        
        # Try to get everything for this patient
        try {
            $everything = Invoke-RestMethod -Uri "$fhirBase/Patient/$patientId/`$everything" -TimeoutSec 30
            $linkedResources = $everything.entry.Count - 1  # Subtract 1 for the patient resource itself
            $results["Patient Graph"] = "$linkedResources linked resources"
            Write-Output "   Patient has $linkedResources linked resources"
        } catch {
            $errorMessage = $_.Exception.Message
            $results["Patient Graph"] = " FAILED: $errorMessage"
            Write-Output "   Could not retrieve patient graph: $errorMessage"
        }
    } else {
        $results["Sample Patient"] = "None found"
        Write-Output "  No patients found in database"
    }
} catch {
    $errorMessage = $_.Exception.Message
    $results["Sample Patient"] = " FAILED: $errorMessage"
    Write-Output "   Error retrieving patients: $errorMessage"
}
"""

print("Running comprehensive HAPI FHIR server health check...")
result = subprocess.run(["powershell", "-NoLogo", "-NoProfile", "-Command", ps_script], 
                        capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr, file=sys.stderr)
    


Running comprehensive HAPI FHIR server health check...

4. Testing Patient query...
   Successfully retrieved patient with ID: 1
   Patient has 19 linked resources



# 4.3 Check database status

In [ ]:
# 5. Check database status
import subprocess, sys

ps_script = r"""
$fhirBase = "http://localhost:8080/fhir"
$results = [ordered]@{}

Write-Output "`n5. Checking database status..."
try {
    # Try to get server status
    if ((Get-Command "docker" -ErrorAction SilentlyContinue)) {
        $stats = docker stats hapi --no-stream --format "{{.CPUPerc}}|{{.MemPerc}}|{{.MemUsage}}"
        $statsArray = $stats -split "\|"
        if ($statsArray.Count -eq 3) {
            $results["CPU Usage"] = $statsArray[0]
            $results["Memory Usage"] = "$($statsArray[1]) ($($statsArray[2]))"
            Write-Output "   Server resources: CPU: $($statsArray[0]), Memory: $($statsArray[1]) ($($statsArray[2]))"
        }
    } else {
        Write-Output " Docker command not available, skipping resource check"
    }
} catch {
    $errorMessage = $_.Exception.Message
    Write-Output " Could not check server resources: $errorMessage"
}
"""

print("Running comprehensive HAPI FHIR server health check...")
result = subprocess.run(["powershell", "-NoLogo", "-NoProfile", "-Command", ps_script], 
                        capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr, file=sys.stderr)


Running comprehensive HAPI FHIR server health check...

5. Checking database status...
   Server resources: CPU: 0.26%, Memory: 29.75% (4.632GiB / 15.57GiB)

========== SUMMARY ==========
HAPI FHIR Server status: OPERATIONAL
{
    "CPU Usage":  "0.26%",
    "Memory Usage":  "29.75% (4.632GiB / 15.57GiB)",
    "Overall Status":  "OPERATIONAL"
}



In [ ]:
import subprocess, sys
ps_script = r"""
$fhirBase = "http://localhost:8080/fhir"
$results = [ordered]@{}

# 6. Overall status
$overallStatus = if ($results["Basic Connectivity"] -like "*") { "OPERATIONAL" } else { "NOT OPERATIONAL" }
$results["Overall Status"] = $overallStatus

Write-Output "`n========== SUMMARY =========="
Write-Output "HAPI FHIR Server status: $overallStatus"

# Return the results object for further processing
$results | ConvertTo-Json -Depth 3
"""

print("Running comprehensive HAPI FHIR server health check...")
result = subprocess.run(["powershell", "-NoLogo", "-NoProfile", "-Command", ps_script], 
                        capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr, file=sys.stderr)

# 4.4 File‑to‑server parity

In [ ]:
import subprocess, textwrap, sys

ps_script = textwrap.dedent(r"""
    $FHIR_BASE = "http://localhost:8080/fhir/"
    Write-Output "Retrieving resource counts via standard search..."
    
    # Common FHIR resource types in Synthea data
    $resourceTypes = @(
        'Patient', 'Practitioner', 'Organization',
        'Encounter', 'Condition', 'Observation', 
        'Procedure', 'MedicationRequest', 'Immunization',
        'AllergyIntolerance', 'CarePlan', 'DiagnosticReport',
        'Goal', 'Medication'
    )
    
    $resourceCounts = [ordered]@{}
    $total = 0
    
    foreach ($type in $resourceTypes) {
        try {
            Write-Output "Checking $type resources..."
            $response = Invoke-RestMethod -Uri "$FHIR_BASE$type`?_summary=count" -Method Get -TimeoutSec 30
            $count = $response.total
            $resourceCounts[$type] = $count
            $total += $count
            Write-Output "  -> Found $count $type resources"
        } catch {
            $errorMessage = $_.Exception.Message
            Write-Output "  -> Error checking $type - $errorMessage"
            $resourceCounts[$type] = 0
        }
    }
    
    $resourceCounts["TOTAL"] = $total
    Write-Output "`nTOTAL RESOURCES: $total"
    
    # Output JSON summary
    $resourceCounts | ConvertTo-Json -Depth 2
""")

print("Retrieving resource counts from server...")
result = subprocess.run(["powershell", "-NoLogo", "-NoProfile", "-Command", ps_script],
                        capture_output=True, text=True)

print(result.stdout)
if result.stderr:
    print("STDERR:", result.stderr, file=sys.stderr)

Retrieving resource counts from server...
Retrieving resource counts via standard search...
Checking Patient resources...
   Found 404 Patient resources
Checking Practitioner resources...
   Found 815 Practitioner resources
Checking Organization resources...
   Found 486 Organization resources
Checking Encounter resources...
   Found 14346 Encounter resources
Checking Condition resources...
   Found 10365 Condition resources
Checking Observation resources...
   Found 77224 Observation resources
Checking Procedure resources...
   Found 40430 Procedure resources
Checking MedicationRequest resources...
   Found 5907 MedicationRequest resources
Checking Immunization resources...
   Found 6011 Immunization resources
Checking AllergyIntolerance resources...
   Found 337 AllergyIntolerance resources
Checking CarePlan resources...
   Found 1212 CarePlan resources
Checking DiagnosticReport resources...
   Found 24073 DiagnosticReport resources
Checking Goal resources...
   Found 0 

# 4.3 Spot‑check a patient graph

In [24]:
import subprocess

# Define PowerShell script to check a random patient's data
ps_script = """
$FHIR_BASE = "http://localhost:8080/fhir/"
$id = (Invoke-RestMethod -Uri ($FHIR_BASE + "Patient?_count=1")).entry[0].resource.id
Write-Output "Random Patient ID = $id"
$everything = Invoke-RestMethod -Uri ($FHIR_BASE + "Patient/$id/`$everything")
Write-Output ("Resources linked to patient: " + $everything.entry.Count)
"""

# Execute PowerShell script
result = subprocess.run(["powershell", "-Command", ps_script], capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Error:", result.stderr)

Random Patient ID = 1
Resources linked to patient: 20



## 5  Next steps
* Enable `$export` and time how long NDJSON generation takes.
* Switch persistence to PostgreSQL for multi‑million‑resource load.
* Layer on SMART‑on‑FHIR or OAuth if you need auth.

---
*Notebook generated automatically from our ChatGPT session (June 2025).*